## libraries and function 

In [1]:
!pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time 

     |████████████████████████████████| 23.1 MB 1.8 MB/s 
     |████████████████████████████████| 2.7 MB 42.1 MB/s 
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=3010618e32a1d093c360582d92bd1ddcd5971684faa307bb026ae74e194a8119
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built knnimpute
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


### MLE estimation function 

In [2]:
def diag_term(X,i):
  arr0 = X[:,i].flatten()
  arr = arr0[~np.isnan(arr0)]
  return np.var(arr)

def musMLE(X,y,G):
    n,p = X.shape[0], X.shape[1]
    f = lambda g: np.nanmean(X[y==g,:],axis=0)
    musMLE = np.array([f(g) for g in range(G)])    
    return musMLE.T

def Smle(X,y,musMLE,g):
    '''
    function to compute the covariance matrix for the g-th class
    X: input, should be a numpy array
    y: label
    G: number of classes
    g: class index
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    Xg, yg = X[y==g,:], y[y==g]
    n,p = Xg.shape[0], Xg.shape[1] 
 
    S = np.diag([diag_term(Xg,i) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        if ((S[i,i] == 0.) | (S[j,j] == 0.)):
          S[i,j] = S[j,i] = 0.
          continue

        mat = Xg[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], yg[idx]
        A = mg = len(y_arr) 

        s11 = mg*np.var(mat[:,0])
        s22 = mg*np.var(mat[:,1])
        s12 = sum((mat[:,0]-musMLE[i,g])*(mat[:,1]-musMLE[j,g]))
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)
        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0
 
        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1: 
          if sum(r==0.0) == len(r):
            r = 0.
          else:  
            w = np.cov(mat, rowvar=False)  
            #r = r[w[0,1]*r>=0]
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 

        S[i,j] = S[j,i] = r
    return S

### compute_err function 

In [3]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [4]:
def compute_err_mle(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    # MLEs approach
    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start  
    e_rate.append(mle_err)
    running_time.append(mle_time)
  e_rate = np.asarray(e_rate)
  running_time = np.asarray(running_time)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)  

In [5]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    print('mus soft impute', mus_softimpute)
    print('S soft impute', S_softimpute)
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start
    running_time.append(softimpute_time)
    e_rate.append(softimpute_err)
  running_time = np.asarray(running_time)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time) 

In [6]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])

    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start
    running_time.append(mice_time)
    e_rate.append(mice_err)
  running_time = np.asarray(running_time)
  e_rate = np.asarray(e_rate)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)

# Digits

In [7]:
digits = datasets.load_digits()
X,y = digits.data, digits.target.ravel() 
print(X.shape)
rmid = np.where(sum(X!=0)<10)
X = np.delete(X, rmid,axis = 1)
X.shape

(1797, 64)


(1797, 54)

In [8]:
G = 10
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid

array([[1.000e-03, 0.000e+00, 2.902e+00],
       [1.000e-03, 0.000e+00, 2.906e+00],
       [1.000e-03, 0.000e+00, 2.927e+00],
       [2.000e-03, 0.000e+00, 2.902e+00],
       [2.000e-03, 0.000e+00, 2.915e+00]])

In [ ]:
G = 10
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

array([[1.000e-03, 0.000e+00, 8.465e+00],
       [1.000e-03, 0.000e+00, 7.923e+00],
       [2.000e-03, 0.000e+00, 8.065e+00],
       [3.000e-03, 0.000e+00, 6.610e+00],
       [4.000e-03, 0.000e+00, 1.195e+01]])

In [ ]:
G = 10
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Max Singular Value of X_init = 95.445104
[SoftImpute] Iter 1: observed MAE=0.035393 rank=54
[SoftImpute] Iter 2: observed MAE=0.035474 rank=54
[SoftImpute] Iter 3: observed MAE=0.035550 rank=54
[SoftImpute] Iter 4: observed MAE=0.035621 rank=54
[SoftImpute] Iter 5: observed MAE=0.035686 rank=54
[SoftImpute] Iter 6: observed MAE=0.035745 rank=54
[SoftImpute] Iter 7: observed MAE=0.035799 rank=54
[SoftImpute] Iter 8: observed MAE=0.035848 rank=54
[SoftImpute] Iter 9: observed MAE=0.035892 rank=54
[SoftImpute] Iter 10: observed MAE=0.035931 rank=54
[SoftImpute] Iter 11: observed MAE=0.035967 rank=54
[SoftImpute] Iter 12: observed MAE=0.035998 rank=54
[SoftImpute] Iter 13: observed MAE=0.036026 rank=54
[SoftImpute] Iter 14: observed MAE=0.036050 rank=54
[SoftImpute] Iter 15: observed MAE=0.036071 rank=54
[SoftImpute] Iter 16: observed MAE=0.036089 rank=54
[SoftImpute] Iter 17: observed MAE=0.036105 rank=54
[SoftImpute] Iter 18: observed MAE=0.036118 rank=54
[SoftImpute] Iter 1

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Iter 3: observed MAE=0.035263 rank=54
[SoftImpute] Iter 4: observed MAE=0.035332 rank=54
[SoftImpute] Iter 5: observed MAE=0.035396 rank=54
[SoftImpute] Iter 6: observed MAE=0.035454 rank=54
[SoftImpute] Iter 7: observed MAE=0.035507 rank=54
[SoftImpute] Iter 8: observed MAE=0.035555 rank=54
[SoftImpute] Iter 9: observed MAE=0.035598 rank=54
[SoftImpute] Iter 10: observed MAE=0.035636 rank=54
[SoftImpute] Iter 11: observed MAE=0.035671 rank=54
[SoftImpute] Iter 12: observed MAE=0.035701 rank=54
[SoftImpute] Iter 13: observed MAE=0.035728 rank=54
[SoftImpute] Iter 14: observed MAE=0.035752 rank=54
[SoftImpute] Iter 15: observed MAE=0.035772 rank=54
[SoftImpute] Iter 16: observed MAE=0.035790 rank=54
[SoftImpute] Iter 17: observed MAE=0.035805 rank=54
[SoftImpute] Iter 18: observed MAE=0.035818 rank=54
[SoftImpute] Iter 19: observed MAE=0.035829 rank=54
[SoftImpute] Iter 20: observed MAE=0.035838 rank=54
[SoftImpute] Iter 21: observed MAE=0.035845 rank=54
[SoftImpute] Iter 2

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Iter 3: observed MAE=0.029199 rank=54
[SoftImpute] Iter 4: observed MAE=0.029208 rank=54
[SoftImpute] Iter 5: observed MAE=0.029217 rank=54
[SoftImpute] Iter 6: observed MAE=0.029225 rank=54
[SoftImpute] Iter 7: observed MAE=0.029234 rank=54
[SoftImpute] Iter 8: observed MAE=0.029241 rank=54
[SoftImpute] Iter 9: observed MAE=0.029248 rank=54
[SoftImpute] Iter 10: observed MAE=0.029255 rank=54
[SoftImpute] Iter 11: observed MAE=0.029261 rank=54
[SoftImpute] Iter 12: observed MAE=0.029267 rank=54
[SoftImpute] Iter 13: observed MAE=0.029273 rank=54
[SoftImpute] Iter 14: observed MAE=0.029278 rank=54
[SoftImpute] Iter 15: observed MAE=0.029283 rank=54
[SoftImpute] Iter 16: observed MAE=0.029287 rank=54
[SoftImpute] Iter 17: observed MAE=0.029291 rank=54
[SoftImpute] Iter 18: observed MAE=0.029294 rank=54
[SoftImpute] Iter 19: observed MAE=0.029297 rank=54
[SoftImpute] Iter 20: observed MAE=0.029300 rank=54
[SoftImpute] Iter 21: observed MAE=0.029303 rank=54
[SoftImpute] Iter 2

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Iter 4: observed MAE=0.029037 rank=54
[SoftImpute] Iter 5: observed MAE=0.029047 rank=54
[SoftImpute] Iter 6: observed MAE=0.029055 rank=54
[SoftImpute] Iter 7: observed MAE=0.029063 rank=54
[SoftImpute] Iter 8: observed MAE=0.029071 rank=54
[SoftImpute] Iter 9: observed MAE=0.029079 rank=54
[SoftImpute] Iter 10: observed MAE=0.029086 rank=54
[SoftImpute] Iter 11: observed MAE=0.029093 rank=54
[SoftImpute] Iter 12: observed MAE=0.029099 rank=54
[SoftImpute] Iter 13: observed MAE=0.029105 rank=54
[SoftImpute] Iter 14: observed MAE=0.029110 rank=54
[SoftImpute] Iter 15: observed MAE=0.029115 rank=54
[SoftImpute] Iter 16: observed MAE=0.029119 rank=54
[SoftImpute] Iter 17: observed MAE=0.029124 rank=54
[SoftImpute] Iter 18: observed MAE=0.029127 rank=54
[SoftImpute] Iter 19: observed MAE=0.029131 rank=54
[SoftImpute] Iter 20: observed MAE=0.029134 rank=54
[SoftImpute] Iter 21: observed MAE=0.029137 rank=54
[SoftImpute] Iter 22: observed MAE=0.029139 rank=54
[SoftImpute] Iter 

array([[1.000e-03, 0.000e+00, 1.201e+00],
       [2.000e-03, 0.000e+00, 1.604e+00],
       [3.000e-03, 0.000e+00, 1.852e+00],
       [4.000e-03, 0.000e+00, 1.924e+00],
       [5.000e-03, 0.000e+00, 1.956e+00]])